In [1]:
import pandas as pd
import numpy as np
import optuna
import logging
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder

# Constants
ENV = 'Kaggle'  # Set to 'Colab', 'Kaggle', or 'Sagemaker'
DEV = False  # Set to True to enable subsetting, False for full training data
SUBSET_SIZE = 1000  # Number of samples for the subset during development
TRIALS = 15  # Number of trials for Optuna

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Basic setup
ID_COL = 'id'
TARGET_COL = 'Depression'

def load_data():
    """Load data based on environment setting."""
    paths = {
        'Kaggle': '/kaggle/input/playground-series-s4e11/',
        'Sagemaker': '/home/ec2-user/SageMaker/data/PS4E11/',
        'Colab': '/content/drive/MyDrive/Kaggle_analysis/PS4E11/data/'
    }
    base_path = paths.get(ENV)
    if not base_path:
        raise ValueError("Invalid environment specified")
    
    train_data = pd.read_csv(base_path + 'train.csv')
    test_data = pd.read_csv(base_path + 'test.csv')
    sample_submission = pd.read_csv(base_path + 'sample_submission.csv')
    
    return train_data, test_data, sample_submission

def preprocess_data(train_data, test_data):
    """Preprocess the data for model training."""
    X = train_data.drop(columns=[ID_COL, TARGET_COL])
    y = train_data[TARGET_COL]
    X_test = test_data.drop(columns=[ID_COL])

    if DEV:
        subset_indices = np.random.choice(X.index, size=min(SUBSET_SIZE, len(X)), replace=False)
        X = X.loc[subset_indices]
        y = y.loc[subset_indices]
        X_test = X_test.iloc[:SUBSET_SIZE]

    # Handle categorical features
    cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
    X[cat_features] = X[cat_features].fillna('missing')
    X_test[cat_features] = X_test[cat_features].fillna('missing')

    # Convert categorical features to category dtype
    for col in cat_features:
        X[col] = X[col].astype('category')
        X_test[col] = X_test[col].astype('category')

    # One-hot encode for XGBoost
    X_encoded = pd.get_dummies(X, columns=cat_features)
    X_test_encoded = pd.get_dummies(X_test, columns=cat_features)

    # Align columns
    X_encoded, X_test_encoded = X_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)

    return X, y, X_test, X_encoded, X_test_encoded, cat_features

def get_model(model_type, params):
    """Create model instance based on type."""
    if model_type == 'CatBoost':
        return CatBoostClassifier(**params)
    elif model_type == 'XGBoost':
        return XGBClassifier(**params)
    elif model_type == 'LightGBM':
        return LGBMClassifier(**params)
    else:
        raise ValueError(f"Invalid model type: {model_type}")

def objective(trial, model_type, X, y, cat_features=None, X_encoded=None):
    """Optuna objective function for hyperparameter optimization."""
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    oof_preds = np.zeros(len(X))

    # Suggest hyperparameters based on model type
    if model_type == 'CatBoost':
        params = {
            'iterations': trial.suggest_int('iterations', 100, 1000),
            'depth': trial.suggest_int('depth', 4, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
            'random_strength': trial.suggest_float('random_strength', 1, 20),
            'verbose': 0,
        }
    elif model_type == 'XGBoost':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 4, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'verbosity': 0,
        }
    elif model_type == 'LightGBM':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'num_leaves': trial.suggest_int('num_leaves', 20, 150),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        }

    for train_idx, valid_idx in skf.split(X, y):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        if model_type == 'XGBoost':
            X_train, X_valid = X_encoded.iloc[train_idx], X_encoded.iloc[valid_idx]

        model = get_model(model_type, params)
        
        try:
            if model_type == 'CatBoost':
                model.fit(
                    X_train, y_train,
                    cat_features=cat_features,
                    eval_set=(X_valid, y_valid),
                    early_stopping_rounds=100,
                    verbose=False
                )
            elif model_type == 'XGBoost':
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_valid, y_valid)],
                    early_stopping_rounds=100,
                    verbose=False
                )
            else:  # LightGBM
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_valid, y_valid)],
                    callbacks=[lgb.early_stopping(stopping_rounds=100)]
                )

            oof_preds[valid_idx] = model.predict(X_valid)
        
        except Exception as e:
            logging.error(f"Error in model fitting: {str(e)}")
            raise

    return accuracy_score(y, oof_preds)

def tune_hyperparameters(X, y, model_type, cat_features=None, X_encoded=None):
    """Run Optuna study to find best hyperparameters."""
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, model_type, X, y, cat_features, X_encoded), n_trials=TRIALS)
    print(f"Best hyperparameters for {model_type}: {study.best_params}")
    return study.best_params

def train_and_predict_with_tuned_params(X, y, X_test, model_type, cat_features, params, X_encoded=None, X_test_encoded=None):
    """Train model with tuned parameters and make predictions."""
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    oof_preds = np.zeros(len(X))
    test_preds = np.zeros(len(X_test))

    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y), 1):
        print(f"Training {model_type} - Fold {fold}/5")
        
        if model_type == 'XGBoost':
            X_train, X_valid = X_encoded.iloc[train_idx], X_encoded.iloc[valid_idx]
            X_test_current = X_test_encoded
        else:
            X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
            X_test_current = X_test

        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        model = get_model(model_type, params)
        
        try:
            if model_type == 'CatBoost':
                model.fit(
                    X_train, y_train,
                    cat_features=cat_features,
                    eval_set=(X_valid, y_valid),
                    early_stopping_rounds=100,
                    verbose=False
                )
            elif model_type == 'XGBoost':
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_valid, y_valid)],
                    early_stopping_rounds=100,
                    verbose=False
                )
            else:  # LightGBM
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_valid, y_valid)],
                    callbacks=[lgb.early_stopping(stopping_rounds=100)]
                )

            oof_preds[valid_idx] = model.predict(X_valid)
            test_preds += model.predict(X_test_current) / skf.n_splits

            # Print fold performance
            fold_score = accuracy_score(y_valid, model.predict(X_valid))
            print(f"{model_type} Fold {fold} accuracy: {fold_score:.4f}")
            
        except Exception as e:
            logging.error(f"Error in fold {fold}: {str(e)}")
            raise

    return oof_preds, test_preds

def train_meta_model(oof_preds_list, y):
    """Train meta-model using base model predictions."""
    X_meta = np.column_stack(oof_preds_list)
    meta_model = LogisticRegression()
    meta_model.fit(X_meta, y)
    return meta_model

def generate_final_predictions(meta_model, test_preds_list):
    """Generate ensemble predictions using meta-model."""
    X_meta_test = np.column_stack(test_preds_list)
    final_preds = meta_model.predict(X_meta_test)
    return final_preds

# Load and preprocess data
print("Loading and preprocessing data...")
train_data, test_data, sample_submission = load_data()
X, y, X_test, X_encoded, X_test_encoded, cat_features = preprocess_data(train_data, test_data)

# Tune and train models
print("\nTuning LightGBM...")
best_params_lgbm = tune_hyperparameters(X, y, 'LightGBM', cat_features)

print("\nTuning CatBoost...")
best_params_catboost = tune_hyperparameters(X, y, 'CatBoost', cat_features)

print("\nTuning XGBoost...")
best_params_xgboost = tune_hyperparameters(X, y, 'XGBoost', None, X_encoded)

# Train models and get predictions
print("\nTraining models with tuned parameters...")
oof_lgbm, test_lgbm = train_and_predict_with_tuned_params(
    X, y, X_test, 'LightGBM', cat_features, best_params_lgbm
)
oof_catboost, test_catboost = train_and_predict_with_tuned_params(
    X, y, X_test, 'CatBoost', cat_features, best_params_catboost
)
oof_xgboost, test_xgboost = train_and_predict_with_tuned_params(
    X, y, X_test, 'XGBoost', None, best_params_xgboost, X_encoded, X_test_encoded
)

# Train meta model and generate final predictions
print("\nTraining meta model...")
oof_preds_list = [oof_catboost, oof_xgboost, oof_lgbm]
test_preds_list = [test_catboost, test_xgboost, test_lgbm]

meta_model = train_meta_model(oof_preds_list, y)
final_preds = generate_final_predictions(meta_model, test_preds_list)

# Create and save submission
submission = pd.DataFrame({
    ID_COL: test_data[ID_COL],
    TARGET_COL: final_preds
})
submission.to_csv('submission.csv', index=False)
print("\nFinal ensemble submission file has been saved.")

# Display sample of predictions
print("\nSample of final predictions:")
print(submission.head())

Loading and preprocessing data...


[I 2024-11-13 12:17:03,265] A new study created in memory with name: no-name-ccee4db9-d353-426b-b70d-b219952e18ad



Tuning LightGBM...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 17045, number of negative: 76755
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014611 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 683
[LightGBM] [Info] Number of data points in the train set: 93800, number of used features: 18
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.181716 -> initscore=-1.504762
[LightGBM] [Info] Start training from score -1.504762
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[32]	valid_0's binary_logloss: 0.160791
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 17045, n

[I 2024-11-13 12:17:17,887] Trial 0 finished with value: 0.9366950959488273 and parameters: {'n_estimators': 476, 'num_leaves': 66, 'learning_rate': 0.1833218914547814, 'min_child_samples': 31, 'subsample': 0.883142706467505}. Best is trial 0 with value: 0.9366950959488273.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 17045, number of negative: 76755
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018173 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 683
[LightGBM] [Info] Number of data points in the train set: 93800, number of used features: 18
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.181716 -> initscore=-1.504762
[LightGBM] [Info] Start training from score -1.504762
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[179]	valid_0's binary_logloss: 0.166161
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 17045, number 

...

XGBoost Fold 3 accuracy: 0.9409
Training XGBoost - Fold 4/5


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBoost Fold 4 accuracy: 0.9414
Training XGBoost - Fold 5/5


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBoost Fold 5 accuracy: 0.9389

Training meta model...

Final ensemble submission file has been saved.

Sample of final predictions:
       id  Depression
0  140700           0
1  140701           0
2  140702           0
3  140703           1
4  140704           0
